<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson OpenScale - Initial Setup

## The notebook will configure OpenScale to monitor a machine learning deployment.

### Contents

- [1.0 Install Python Packages](#setup)
- [2.0 Configure Credentials](#credentials)
- [3.0 OpenScale configuration](#openscale)
- [4.0 Create Datamart](#datamart)
- [5.0 Bind Machine Learning engines](#bind)
- [6.0 Check and setup subscriptions](#subscriptions)
- [7.0 Score the model](#score)
- [8.0 Store the variables](#store)

# 1.0 Install Python Packages <a name=setup></a>

In [3]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade ibm-ai-openscale==2.1.21 --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4==1.0.95 | tail -n 1

### Action: restart the kernel!

# 2 .0 Configure credentials <a name="credentials"></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

<font color='red'>Replace the `username` and `password` values of `************` with your Cloud Pak for Data `username` and `password`. The value for `url` should match the `url` for your Cloud Pak for Data cluster, which you can get from the browser address bar (be sure to include the 'https://'.</font> The credentials should look something like this (these are example values, not the ones you will use):

`
WOS_CREDENTIALS = {
                   "url": "https://zen.clusterid.us-south.containers.appdomain.cloud",
                   "username": "cp4duser",
                   "password" : "cp4dpass"
                  }
`
#### NOTE: Make sure that there is no trailing forward slash `/` in the `url`

In [1]:
WOS_CREDENTIALS = {
    "url": "https://************",
    "username": "************",
    "password": "************"
}

In [2]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='3.0.0'

# 3.0 Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and configure OpenScale

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [4]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [5]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

'2.1.21'

# 4.0 Create datamart <a name="datamart"></a>

## 4.1 Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If an OpenScale datamart exists in Db2, the existing datamart will be used and no data will be overwritten.

Prior instances of the Credit model will be removed from OpenScale monitoring.

In [9]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    print('Using existing external datamart')
except:
    print('Setting up external datamart')
    ai_client.data_mart.setup(db_credentials=DATABASE_CREDENTIALS, schema=SCHEMA_NAME)

Using existing external datamart


## 5.0  Bind machine learning engines <a name="bind"></a>

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. 

### Create the binding if it doesn't already exist.

In [11]:
binding_uid = None

binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
if binding_uid is None:
    print("Adding binding:")
    binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance4ICP(wml_credentials=WML_CREDENTIALS))
    bindings_details = ai_client.data_mart.bindings.get_details()
else:
    print("Found binding:")
binding_uid

Found binding:


'405e817b-3715-453b-91a7-6a36186de087'

In [12]:
ai_client.data_mart.bindings.list()

3c617b36-f0f6-406d-899a-2b7086356da7,WML instance,watson_machine_learning,2020-07-16T17:17:59.614Z
f58831a3-3816-4ae5-9552-93500b448475,WML instance,watson_machine_learning,2020-07-10T15:30:48.227Z
262c4a39-0b5c-4787-aba8-7e9efe495445,WML instance,watson_machine_learning,2020-07-09T22:02:03.147Z
3f65b02c-ad55-4e51-bdf3-5bb87275cd04,WML instance,watson_machine_learning,2020-07-05T20:06:43.004Z
507f0e58-5ac1-4f22-bf92-fe51747b1d18,WML instance,watson_machine_learning,2020-06-16T05:32:13.707Z
d45d24c4-113a-470d-9669-2a39ed49faf5,WML instance,watson_machine_learning,2020-06-16T04:34:12.349Z
69eeed33-8d5b-4c4c-950b-97dd7f1c4cca,WML instance,watson_machine_learning,2020-06-16T03:16:56.422Z
3ee275c9-1325-488b-93ec-23a07ad5d54a,WML instance,watson_machine_learning,2020-06-15T22:49:40.887Z
fb3a6090-36eb-412a-a465-6b4ecd1a3bd4,WML instance,watson_machine_learning,2020-06-15T20:30:51.235Z
8c569603-7483-46eb-b722-1652ea5a6d36,WML instance,watson_machine_learning,2020-06-15T16:50:04.080Z
405e817b-3715-453b-91a7-6a36186de087,WML instance,watson_machine_learning,2020-06-15T16:26:24.549Z


### 5.1 get list of assets

In [13]:
ai_client.data_mart.bindings.list_assets()

5b2016f1-ad4e-47f5-af86-32497f372383,sda-model-8-3-2020,2020-08-03T13:16:53.002Z,model,mllib_2.3,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,False
5b2016f1-ad4e-47f5-af86-32497f372383,sda-model-8-3-2020,2020-08-03T13:16:53.002Z,model,mllib_2.3,262c4a39-0b5c-4787-aba8-7e9efe495445,False
011da060-cb3b-4224-8bde-b15004fdab5b,SANDHYA_MODEL,2020-07-16T17:17:01.002Z,model,mllib_2.3,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,True
011da060-cb3b-4224-8bde-b15004fdab5b,SANDHYA_MODEL,2020-07-16T17:17:01.002Z,model,mllib_2.3,262c4a39-0b5c-4787-aba8-7e9efe495445,True
011da060-cb3b-4224-8bde-b15004fdab5b,SANDHYA_MODEL,2020-07-16T17:17:01.002Z,model,mllib_2.3,f58831a3-3816-4ae5-9552-93500b448475,True
011da060-cb3b-4224-8bde-b15004fdab5b,SANDHYA_MODEL,2020-07-16T17:17:01.002Z,model,mllib_2.3,3c617b36-f0f6-406d-899a-2b7086356da7,True
1db1301e-42e2-4f67-ad37-58df6102d961,Trading Customer Churn Prediction Model,2020-07-13T22:30:40.002Z,model,mllib_2.3,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,False
1db1301e-42e2-4f67-ad37-58df6102d961,Trading Customer Churn Prediction Model,2020-07-13T22:30:40.002Z,model,mllib_2.3,262c4a39-0b5c-4787-aba8-7e9efe495445,False
1db1301e-42e2-4f67-ad37-58df6102d961,Trading Customer Churn Prediction Model,2020-07-13T22:30:40.002Z,model,mllib_2.3,f58831a3-3816-4ae5-9552-93500b448475,False
1db1301e-42e2-4f67-ad37-58df6102d961,Trading Customer Churn Prediction Model,2020-07-13T22:30:40.002Z,model,mllib_2.3,3c617b36-f0f6-406d-899a-2b7086356da7,False
927d417e-59d8-4a76-bba3-72fc56a9e5ae,my name Risk Model,2020-07-09T22:01:36.002Z,model,mllib_2.3,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,False


###  5.2 Action: Set the MODEL_NAME to your depoloyed mllib model below:

In [14]:
MODEL_NAME = "sda-model-8-3-2020"

In [15]:
ai_client.data_mart.bindings.get_details(binding_uid)

{'entity': {'credentials': {'instance_id': 'openshift',
   'password': '<redacted>',
   'url': 'https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud',
   'username': 'samaya',
   'version': '3.0.0'},
  'instance_id': '405e817b-3715-453b-91a7-6a36186de087',
  'name': 'WML instance',
  'service_type': 'watson_machine_learning',
  'status': {'state': 'active'}},
 'metadata': {'guid': '405e817b-3715-453b-91a7-6a36186de087',
  'url': '/v1/data_marts/00000000-0000-0000-0000-000000000000/service_bindings/405e817b-3715-453b-91a7-6a36186de087',
  'created_at': '2020-06-15T16:26:24.549Z'}}

## 6.0 Subscriptions <a name="subscriptions"></a>

### Only if needed, remove existing credit risk subscriptions
This code removes previous subscriptions to the Credit model to refresh the monitors with the new model and new data.
This should not be needed and is only removed to cleanup a problem situation.

In [ ]:
# subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
# for subscription in subscriptions_uids:
#    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
#    if sub_name == MODEL_NAME:
#        ai_client.data_mart.subscriptions.delete(subscription)
#        print('Deleted existing subscription for', MODEL_NAME)

###  6.1 Get the list of deployment spaces and use the GUID to set the default_space

In [16]:
wml_client.spaces.list()

------------------------------------  ------------------------  ------------------------
GUID                                  NAME                      CREATED
3f2c0a53-e1bb-4e09-957d-d7b84c16a56f  scottda-deployment-space  2020-08-03T13:15:31.571Z
------------------------------------  ------------------------  ------------------------


### 6.2 Action: We'll use the `GUID` for your Deployment space as listed above to replace `******` for  the `default_space`  below:

In [17]:
default_space = "3f2c0a53-e1bb-4e09-957d-d7b84c16a56f"

In [18]:
wml_client.set.default_space(default_space)

'SUCCESS'

In [19]:
wml_models = wml_client.repository.get_model_details()
model_uid = None

for model_in in wml_models['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break
        
print(model_uid)

5b2016f1-ad4e-47f5-af86-32497f372383


### 6.3 Action:  Set the DEPLOYMENT_NAME
Use the name of the Deployment that is associated with your machine learning model

In [20]:
wml_client.deployments.list()

------------------------------------  ---------------------------------  -----  ------------------------  -------------
GUID                                  NAME                               STATE  CREATED                   ARTIFACT_TYPE
c3d1c135-9792-47bc-9719-946e42da306d  scottda-batch-deployment-8-3-2020  ready  2020-08-03T18:39:24.968Z  model
d435943c-d802-4e3b-b34c-e0f42fa4ee52  scottda-model-deployment-8-3-2020  ready  2020-08-03T17:03:56.054Z  model
------------------------------------  ---------------------------------  -----  ------------------------  -------------


In [21]:
DEPLOYMENT_NAME = "scottda-model-deployment-8-3-2020"

In [22]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break
        
print(deployment_uid)

d435943c-d802-4e3b-b34c-e0f42fa4ee52


## 6.3 This code creates the model subscription in OpenScale using the Python client API. 
> Note that we need to provide the model unique identifier,and some information about the model itself.

### Check to see if subscription already exists, and use it if it does

In [23]:
subscription = None
if subscription is None:
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            print("Found existing subscription")
            subscription = ai_client.data_mart.subscriptions.get(sub)
    if subscription is None:
        print("No subscription found. Run the cell below to add the subscription")

No subscription found. Run the cell below to add the subscription


### If the subscription is not found, add it now

In [25]:
if subscription is None:
    subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(        
        model_uid,
        
        problem_type=ProblemType.BINARY_CLASSIFICATION,
        input_data_type=InputDataType.STRUCTURED,
        label_column='Risk',
        prediction_column='predictedLabel',
        probability_column='probability',
        feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration",
                           "InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing",
                           "ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
        categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan",
                           "OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
    ), deployment_uids=deployment_uid)

Get subscription list

In [26]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

f978ce05-e4f6-42b0-81b2-be06884cd358,sda-model-8-3-2020,model,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,2020-08-04T11:41:26.677Z
6d033614-d278-46e4-a8e6-7511839200df,SANDHYA_MODEL,model,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,2020-07-16T17:19:15.345Z
2ba77ee8-4bc0-4087-b913-e557594850a7,my name Risk Model,model,3f65b02c-ad55-4e51-bdf3-5bb87275cd04,2020-07-10T15:33:24.808Z
c0feeb64-26d5-436d-806a-4954c3235bf0,SAMAYA-DRIFT_MODEL,model,405e817b-3715-453b-91a7-6a36186de087,2020-06-16T05:32:46.827Z


In [27]:
subscription_details = subscription.get_details()

### 7.0 Score the model so we can configure monitors <a name="score"></a>

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [28]:
credit_risk_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        credit_risk_scoring_endpoint = deployment['entity']['status']['online_url']['url']
        
print(credit_risk_scoring_endpoint)

d435943c-d802-4e3b-b34c-e0f42fa4ee52
https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud/v4/deployments/d435943c-d802-4e3b-b34c-e0f42fa4ee52/predictions


In [29]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration",
                           "InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing",
                           "ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'LoanPurpose_IX', 'ExistingSavings_IX', 'EmploymentDuration_IX', 'Sex_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'InstallmentPlans_IX', 'Housing_IX', 'Job_IX', 'Telephone_IX', 'ForeignWorker_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.0, 1.0, 13.0, 

## 8.0 Store the variables <a name="store"></a>
### This will store the important variables for use in future notebooks

In [30]:
DEFAULT_SPACE = default_space 

%store MODEL_NAME
%store DEPLOYMENT_NAME
%store DEFAULT_SPACE
%store model_uid
%store binding_uid

Stored 'MODEL_NAME' (str)
Stored 'DEPLOYMENT_NAME' (str)
Stored 'DEFAULT_SPACE' (str)
Stored 'model_uid' (str)
Stored 'binding_uid' (str)
